<a id="charting-top"></a>
<div style="width: 50px; height: 50px; border-radius: 50%; background: #F1C40F; margin-bottom: 0px; text-align: center;">
    <img src="../img/Welcome Page/launch.png" style="max-height: 95%; max-width: 95%;">
</div>
<h1 style="margin-top: 20px; margin-bottom: 5px;">Charting<br><span style="color: orange; margin-bottom: 0px;">Data Visualization with Python Libraries</span></h1>
<h4 style="color: orange; margin-top: 0px;">(20-min Read)</h4>

By the end of this document, you will be able to:
1. Use BQL and the `pandas` library to format a DataFrame for visualization
1. Build and display a time series Line Plot with `bqviz`
1. Build and display a cross-sectional Scatter Plot with `bqviz`

### Contents
- [The BQuant Project Flow](#ch-flow)
- [BQuant Supported Libraries](#ch-libs)
- [Get Data with BQL](#ch-bqviz)
- [Initialize a Time Series Line Plot](#ch-ts)
- [Initialize a Cross-Sectional Scatter Plot](#ch-scatter)


<a href="2 DataGrids.ipynb">&larr; Back to DataGrids</a>&emsp; | &emsp;
<a href="4 Launch Project.ipynb">Continue to Launch: Final Project &rarr;</a>

---
<a id="ch-flow"></a>
<h1>BQuant Project Flow<br><span style="color: orange;">5-Step Process for Developing Your Projects</span></h1>

The 5-step project flow for BQuant development was mentioned in the Intro to BQuant section, but it bears repeating here. Every project you develop in BQuant will follow the same basic process below.

1. Setup Environment
1. Get Data from Bloomberg using BQL
1. Additional Data Munging (if necessary)
1. User Interface & Visualizations
1. Simplify, Document, & Publish

In this notebook, we cover the use of charting libraries in Step 4.

[&uarr; Return to Top](#charting-top)

---
<a id="ch-libs"></a>
<h1>BQuant Supported Libraries<br><span style="color: orange;">Choose Your Charting Weapon</span></h1>

The BQuant environment supports several charting libraries, both open-source and proprietary to Bloomberg. For newcomers to Python data visualization, we cover the Bloomberg-created `bqviz` library in this tutorial. However, if you have experience in another charting library, feel free to use that in your apps and skip the rest of this tutorial.

The open-source libraries supported by BQuant include the following.
   - [matplotlib](https://matplotlib.org/)
   - [plotly](https://plotly.com/python/)
   - [seaborn](https://seaborn.pydata.org/)
   - [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/) for interactive maps

<br>
Bloomberg's proprietary charting libraries include the following.
<ul>
    <li><a href="../exampleroot/7da582b97da6463aa0bf19221f44b28a/bqplot Basics.ipynb">bqplot</a></li>
    <li><a href="../exampleroot/2f85317198594ff0a9f8bf6cb8898c3d/BQViz Basics.ipynb">bqviz</a> - a wrapper library for <code>bqplot</code> allowing you to quickly create <code>bqplot</code> charts.</li>
</ul>
   

[&uarr; Return to Top](#charting-top)

---
<a id="ch-bqviz"></a>
<h1>Get Data with BQL<br><span style="color: orange;">Use BQL to Prepare Data for Visualization</span></h1>

If you're just getting started with Python data visualization, Bloomberg's `bqviz` library offers a user-friendly syntax and will generate charts with only a few lines of code. Before we can use `bqviz`, however, we need some data to chart! The code cell below uses BQL to retrieve time series and cross-sectional data for us to work with in `bqviz`.

Oftentimes the most complex part of your projects will be the BQL query. Once the data is in the proper format, we can build charts in `bqviz` with just a single line of code. Before we can do that, however, we need to fetch, clean, and prepare DataFrames that `bqviz` chart objects can accept. For time series, `bqviz` works best when the time series dates are in the DataFrame's index, and there are no unrelated columns. For the cross-sectional data, we combine two BQL requests so we can have X and Y axes in a scatter plot.

The successful use of any charting library will depend on how well you clean and prepare your DataFrame before visualization. The BQL query below might seem complicated, but it saves us from writing many lines of code to force the DataFrame into the proper form for `bqviz` charts.

<h5 style="color: orange;">BQL Queries for Time Series & Cross-Sectional Data</h5>

In [1]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
import bql
import pandas as pd
from IPython.display import display
bq = bql.Service()

# Step 2: Get data with BQL
# Time series request
universe = 'B500T Index'
prices = bq.data.px_last('-1Y', '0D', fill='prev')
ts_req = bql.Request(universe, {'Bloomberg US Large Cap Price': prices})
ts_res = bq.execute(ts_req)
ts_df = ts_res[0].df().set_index('DATE')
ts_df.drop('CURRENCY', inplace=True, axis=1)

# Cross-Sectional request
# get average FCF and Sales Growth by sector
sector = bq.data.gics_sector_name()
b500_members = bq.univ.members(universe)
fcf_growth = bq.data.free_cash_flow_1_year_growth() / 100
rev_growth = bq.data.sales_growth() / 100
sector_avg_fcf_growth = fcf_growth.group(sector).avg()
sector_avg_rev_growth = rev_growth.group(sector).avg()
cs_req = bql.Request(b500_members, {'Avg FCF Growth': sector_avg_fcf_growth,
                                     'Avg Sales Growth': sector_avg_rev_growth})
cs_res = bq.execute(cs_req)
cs_df = pd.concat([field.df()[field.name] for field in cs_res], axis=1).reset_index()

# preview the results
print('Time Series')
display(ts_df.head())

print('\nCross-Sectional')
display(cs_df.head())

Time Series


,Bloomberg US Large Cap Price
DATE,
2020-04-08,1009.03
2020-04-09,1023.79
2020-04-10,1023.79
2020-04-11,1023.79
2020-04-12,1023.79



Cross-Sectional


,ID,Avg FCF Growth,Avg Sales Growth
0,Communication Services,0.141390,0.198956
1,Consumer Discretionary,1.612408,0.035429
2,Consumer Staples,0.335794,0.035997
3,Energy,-0.022364,-0.260683
4,Financials,0.279426,0.031316


[&uarr; Return to Top](#charting-top)

---
<a id="ch-ts"></a>
<h1>Initialize a Time Series Line Plot<br><span style="color: orange;">Explore <code style="color: orange; background-color: transparent;">bqviz.LinePlot()</code></span></h1>

Now that we have some data to work with, we can explore the charting options available to us in `bqviz`. For many chart objects in the `bqviz` library, the only required variable is a DataFrame, meaning that we can initiate and display a chart very quickly. The `set_style()` method will apply a default styling to the chart, and the `show()` method will render the chart to the screen.

The example below prepares and displays a time series Line Plot for our S&P 500 prices.

In [ ]:
# Run this cell with the Run button in the Jupyter toolbar
# import the library
import bqviz

bqviz.LinePlot(ts_df).set_style().show()

There's more to the `LinePlot` object than is covered in this tutorial! Check the <a href="../exampleroot/2f85317198594ff0a9f8bf6cb8898c3d/BQViz Basics.ipynb">bqviz documentation</a> to explore customization & multiple-series options.

<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice initializing a <code>bqviz.LinePlot</code>
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Write a BQL query that retrieves the last 30 days of prices for two securities of your choice.</li>
            <li>Use BQL and the <code>pandas</code> library to format the DataFrame so that dates are in the index column and the only other columns are security price data. Reference the previous <a href="../1. Boot Camp/3 BQL Basics.ipynb#bql=example3">Multiple Universes, One Data Item</a> BQL Example to help you format the DataFrame.</li>
            <li>Declare a variable named <code>my_ts_df</code> to store the resulting DataFrame.</li>
            <li>Run <code>bqviz.LinePlot(my_ts_df).set_style().show()</code> to view your data in a LinePlot.</li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# import bqviz
# import bql
# import pandas as pd
# bq = bql.Service()

# BQL Query
universe = ['9988 HK Equity', '700 HK Equity']    # <-- Alibaba & Tencent
prices = bq.data.px_last('-30D', '0D').dropna()   # <-- drop non-trading days
request = bql.Request(universe, {'Closing Price': prices})
response = bq.execute(request)
# parse the data so that each security has its own column
my_ts_df = response[0].df().reset_index().pivot(index='DATE', values='Closing Price', columns='ID')

bqviz.LinePlot(my_ts_df).set_style().show()

[&uarr; Return to Top](#charting-top)

---
<a id="ch-scatter"></a>
<h1>Initialize a Cross-Sectional Scatter Plot<br><span style="color: orange;">Explore <code style="color: orange; background-color: transparent;">bqviz.ScatterPlot()</code></span></h1>

Since we made the effort to correctly format our cross-sectional DataFrame, viewing it in a `bqviz.ScatterPlot` can be done in just a few lines of code. The `ScatterPlot` requires a few additional arguments.

  - `x` - column name for the independent variable
  - `y` - column name for the dependent variable 
  - `labels` - while not required, specifying a column name for marker labels will help users understand your plot
  - `display_labels` - boolean should be set to `True` for labels to be displayed

In [46]:
# Run this cell with the Run button in the Jupyter toolbar
scatter = bqviz.ScatterPlot(df=cs_df,
                            x='Avg Sales Growth', 
                            y='Avg FCF Growth', 
                            labels='ID', 
                            display_labels=True)

scatter.set_style().show()

GridBox(children=(Figure(animation_duration=500, axes=[Axis(color='white', grid_color='#3c3c3c', grid_lines='d…

There's more to the `ScatterPlot` object than is covered in this tutorial! Check the <a href="../exampleroot/2f85317198594ff0a9f8bf6cb8898c3d/BQViz Basics.ipynb">bqviz documentation</a> to explore customization options.

<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice Initializing a <code>bqviz.ScatterPlot</code>
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Write a BQL query that retrieves average <code>PE_RATIO(fill='prev')</code> and <code>PX_TO_SALES_RATIO(fill='prev')</code> ratios by sector for members of the Bloomberg Hong Kong Index (<code>'HKLSPL Index'</code>)</li>
            <li>Use BQL and the <code>pandas</code> library to format the DataFrame so that it has three columns: the name of the sector, P/E Ratios, and P/S Ratios. Reference the previous <a href="../1. Boot Camp/3 BQL Basics.ipynb#df-concat">Concatenate, Merge, & Pivot</a> BQL Examples to help you format the DataFrame. Since this is cross-sectional data, a DataFrame concatenation with <code>pd.concat()</code> may be easier to implement than a DataFrame merge. Use the BQL query in the <a href="#ch-bqviz">Get BQL Data</a> section as a guide.</li>
            <li>Declare a variable named <code>my_cs_df</code> to store the resulting DataFrame.</li>
            <li>Build and show the ScatterPlot</li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# import bql
# import bqviz
# bq = bql.Service()

universe = bq.univ.members('HKLSPL Index')
pe_ratio = bq.data.pe_ratio(fill='PREV')
ps_ratio = bq.data.px_to_sales_ratio(fill='PREV')
company_name = bq.data.name()  # <-- get company name for scatter plot labels
request = bql.Request(universe, {'PE Ratio': pe_ratio,
                                 'PS Ratio': ps_ratio,
                                 'Name': company_name})
response = bq.execute(request)
# parse & clean dataframes
pe_df = response[0].df()
ps_df = response[1].df()
name_df = response[2].df()
df = pd.concat([pe_df, ps_df, name_df], axis=1)

bqviz.ScatterPlot(df=df, x='PE Ratio', y='PS Ratio', labels='Name').set_style().show()

[&uarr; Return to Top](#charting-top)

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="2 DataGrids.ipynb">&larr; Back to DataGrids</a>&emsp;&emsp;
    <a href="#charting-top">&uarr; Return to Top</a>&emsp;&emsp;
    <a href="4 Launch Project.ipynb">Continue to Launch: Final Project &rarr;</a>
    <br>
    <br>
    <a href="../Welcome.ipynb#welcome-top" style="font-size: 12px;">Return to the Welcome Page</a>
</p>